In [113]:
import copy
import time
import math
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from tqdm import tqdm
from datetime import datetime
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [114]:
device = torch.device('cpu')

appa1 = pd.read_csv("noam_exports/appa1.csv")
appa1 = appa1.drop(columns='Unnamed: 0')

# CNN-LSTM Neural Network

In [142]:
from torch import max_pool2d


class CNN_LSTM(nn.Module):
    def __init__(self):
        super(CNN_LSTM, self).__init__()
        self.conv = nn.Conv2d(in_channels=1, out_channels=40, kernel_size=(4, 37))
        self.maxpool = nn.MaxPool2d(kernel_size=(4, 37))
        self.lstm = nn.LSTM(15, 200)
        self.fc1 = nn.Linear(200, 150)
        self.fc2 = nn.Linear(150, 50)
        self.fc3 = nn.Linear(50,3)
        self.dropout1 = nn.Dropout(0.03)
        self.dropout2 = nn.Dropout(0.1)
        self.dropout3 = nn.Dropout(0.1)
    
    def forward(self, x):
        x = self.conv(x)
        x = self.maxpool(x)
        x = F.relu(x)
        x = torch.squeeze(x)
        print(x.shape)
        print(x.size(-1))
        x, (h_n, c_n) = self.lstm(x, (torch.zeros(2, 15, 200), torch.zeros(2, 15, 200)))
        x = F.relu(x)
        x = self.fc1(x)
        x = self.dropout1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = self.dropout2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = self.dropout3(x)
        return x

CNN_LSTM = CNN_LSTM()
print(CNN_LSTM)

CNN_LSTM(
  (conv): Conv2d(1, 40, kernel_size=(4, 37), stride=(1, 1))
  (maxpool): MaxPool2d(kernel_size=(4, 37), stride=(4, 37), padding=0, dilation=1, ceil_mode=False)
  (lstm): LSTM(15, 200)
  (fc1): Linear(in_features=200, out_features=150, bias=True)
  (fc2): Linear(in_features=150, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=3, bias=True)
  (dropout1): Dropout(p=0.03, inplace=False)
  (dropout2): Dropout(p=0.1, inplace=False)
  (dropout3): Dropout(p=0.1, inplace=False)
)


In [116]:
X = appa1[[
       'ZnOR_1', 'ZnOR_2',
       'LaFeO3_1', 'LaFeO3_2',
       'WO3_1', 'WO3_2',
       'ZnOR_1_heatR', 'ZnOR_2_heatR',
       'LaFeO3_1_heatR', 'LaFeO3_2_heatR',
       'WO3_1_heatR', 'WO3_2_heatR',
       'ZnOR_1_heatV', 'ZnOR_2_heatV',
       'LaFeO3_1_heatV', 'LaFeO3_2_heatV',
       'WO3_1_heatV', 'WO3_2_heatV',
       'Temperature', 'Relative_Humidity', 'Pressure', 'VOC',
       'ZnOR_1_Age', 'ZnOR_2_Age',
       'LaFeO3_1_Age', 'LaFeO3_2_Age',
       'WO3_1_Age', 'WO3_2_Age',
       'sin_hour', 'cos_hour',
       'sin_weekday', 'cos_weekday',
       'sin_month', 'cos_month',
       'sin_ordate', 'cos_ordate',
       'year'
    ]].to_numpy()
Y = appa1[['NO2', 'O3', 'CO']].to_numpy()

In [117]:
X = np.lib.stride_tricks.sliding_window_view(X, (24,37))
Y = Y[X.shape[2]-1:]
(X.shape,Y.shape)

((13678, 1, 24, 37), (13678, 3))

In [118]:
train, test = train_test_split(list(zip(X,Y)), test_size=0.2)

train_dl = DataLoader(train, batch_size=15, shuffle=True)
test_dl = DataLoader(test, batch_size=15, shuffle=True)

In [119]:
def fit_model(model):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = CNN_LSTM
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    for epoch in range(1, 11):
        train_loss = 0
        for features, tag in tqdm(train_dl):
            optimizer.zero_grad()
            pred = model(torch.tensor(features).float())
            loss = criterion(pred, tag)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        print(f'EPOCH: {epoch}; TRAIN LOSS: {train_loss}')
        test_loss = 0
        with torch.no_grad():
            for features, tag in tqdm(test_dl):
                pred = model(tag)
                loss = criterion(pred, tag)
                test_loss += loss.item()
                print(f'EPOCH: {epoch}; TEST LOSS: {test_loss}')

    torch.cuda.empty_cache()

In [143]:
fit_model(CNN_LSTM)

  0%|          | 0/730 [00:00<?, ?it/s]C:\Users\nzams\AppData\Local\Temp\ipykernel_12912\1177280461.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred = model(torch.tensor(features).float())
  0%|          | 0/730 [00:00<?, ?it/s]


RuntimeError: Given input size: (40x21x1). Calculated output size: (40x5x0). Output size is too small